In [1]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, random_split
from UCroma import PretrainedCROMA
import copy

from CR_LO import Loader
from CR_FE import FExtractor
from TR import Trainer
from TE import Tester


sample_size = 120
batch_size = 16

Croma = PretrainedCROMA(pretrained_path='CR.pt', size='base', modality='both', image_resolution=sample_size)
num_features = Croma(SAR_images=torch.randn(1, 2, sample_size, sample_size), 
                     optical_images=torch.randn(1, 12, sample_size, sample_size))['joint_GAP'].shape[1]
head = nn.Sequential(nn.Linear(num_features, 1000), nn.ReLU(), nn.Linear(1000, 1000), nn.ReLU(), nn.Linear(1000, 2))

opt_root_dir = "/home/stagiaire/D/R/patchs/90"
sar_root_dir = "/home/stagiaire/D/R/patchs/90R"

num_cv = 5
num_folds = 5
metrics = [[] for _ in range(num_cv)]

for n in range(num_cv):
    processor = Loader(opt_root_dir=opt_root_dir, sar_root_dir=sar_root_dir, num_folds=num_folds)
    processor.load_data(sample_size=sample_size, batch_size=batch_size, data_seed=None, split_seed=None)
    loaders = processor.loaders
    
    floaders = []
    for loader in loaders:
        feature_extractor = FExtractor(dataloader=loader, use_8_bit=True)
        features, labels = feature_extractor.extract_features()
        dataset = TensorDataset(torch.from_numpy(features), torch.from_numpy(labels))
        floaders.append(DataLoader(dataset, batch_size=batch_size, shuffle=True))
        
    for i in range(num_folds):
        
        head_i = copy.deepcopy(head)
        
        train_loaders = [floaders[(j + i) % num_folds] for j in range(num_folds - 1)]
        test_loader = floaders[(i + num_folds - 1) % num_folds]
        
        for train_loader in train_loaders:
            trainer = Trainer(head_i, train_loader)
            trainer.train(num_epochs=100)
            
        tester = Tester(head_i, test_loader, processor.combined_dataset)
        tester.evaluate()
        
        metrics[n].append(tester.report)

/home/stagiaire/miniconda3/envs/croma/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/stagiaire/miniconda3/envs/croma/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSs'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder
Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


Extracting Features: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.44it/s]


Initializing SAR encoder
Initializing optical encoder
Initializing joint SAR-optical encoder


Extracting Features: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:02<00:00,  5.55it/s]


...


Jaccard index:  81.8%


                Predicted Class 0  Predicted Class 1
Actual Class 0                 74                 18
Actual Class 1                  2                 90


              precision    recall  f1-score   support

           0       0.97      0.80      0.88        92
           1       0.83      0.98      0.90        92

    accuracy                           0.89       184
   macro avg       0.90      0.89      0.89       184
weighted avg       0.90      0.89      0.89       184




Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 88.44 epochs/s]



Jaccard index:  70.9%


                Predicted Class 0  Predicted Class 1
Actual Class 0                 74                 18
Actual Class 1                 14                 78


              precision    recall  f1-score   support

           0       0.84      0.80      0.82        92
           1       0.81      0.85      0.83        92

    accuracy                           0.83       184
   macro avg       0.83      0.83      0.83       184
weighted avg       0.83      0.83      0.83       184




Training: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:01<00:00, 89.36 epochs/s]


Jaccard index:  66.3%


                Predicted Class 0  Predicted Class 1
Actual Class 0                 79                 25
Actual Class 1                 10                 69


              precision    recall  f1-score   support

           0       0.89      0.76      0.82       104
           1       0.73      0.87      0.80        79

    accuracy                           0.81       183
   macro avg       0.81      0.82      0.81       183
weighted avg       0.82      0.81      0.81       183




In [2]:
from R import RGenerator
    
new_metrics = []
for l in metrics:
    new_metrics.extend(l)

reporter = RGenerator(new_metrics).report()

Precision (mean, stdev): 84%, 5%
Recall (mean, stdev): 85%, 6%
F1-score (mean, stdev): 84%, 3%
IoU (mean, stdev): 73%, 4%

[[77. 15.]
 [14. 78.]]

[[4. 5.]
 [6. 5.]]



In [3]:
for l in metrics:
    reporter = RGenerator(l).report()

Precision (mean, stdev): 84%, 5%
Recall (mean, stdev): 85%, 7%
F1-score (mean, stdev): 84%, 1%
IoU (mean, stdev): 73%, 2%

[[77. 15.]
 [15. 77.]]

[[4. 6.]
 [7. 4.]]

Precision (mean, stdev): 86%, 3%
Recall (mean, stdev): 86%, 5%
F1-score (mean, stdev): 86%, 2%
IoU (mean, stdev): 75%, 2%

[[79. 13.]
 [13. 79.]]

[[4. 3.]
 [5. 4.]]

Precision (mean, stdev): 82%, 5%
Recall (mean, stdev): 82%, 6%
F1-score (mean, stdev): 82%, 3%
IoU (mean, stdev): 69%, 4%

[[75. 17.]
 [16. 75.]]

[[1. 5.]
 [6. 4.]]

Precision (mean, stdev): 84%, 4%
Recall (mean, stdev): 84%, 4%
F1-score (mean, stdev): 84%, 3%
IoU (mean, stdev): 72%, 4%

[[77. 15.]
 [14. 77.]]

[[5. 4.]
 [4. 5.]]

Precision (mean, stdev): 83%, 6%
Recall (mean, stdev): 86%, 7%
F1-score (mean, stdev): 84%, 3%
IoU (mean, stdev): 73%, 5%

[[76. 16.]
 [13. 79.]]

[[3. 6.]
 [7. 7.]]

